<h1> Groupe 3 - DashBoard
    
> Script Insertion BD MySQL Notebook

<span class="tocSkip"></span>

> *Authors M2 : Quentin MARCU, Thomas ROY, Rania YAHYAOUI* <br>
> *Authors M1 : Alice AIRAULT, Thomas ZEUTZIUS* <br>
> *Author L3 : Théo SACCAREAU*

</h1>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Tables-dimension" data-toc-modified-id="Tables-dimension-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tables dimension</a></span><ul class="toc-item"><li><span><a href="#AIRLINE" data-toc-modified-id="AIRLINE-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>AIRLINE</a></span></li><li><span><a href="#ROUTE" data-toc-modified-id="ROUTE-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ROUTE</a></span></li><li><span><a href="#SERVICE" data-toc-modified-id="SERVICE-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>SERVICE</a></span></li><li><span><a href="#PASSENGER" data-toc-modified-id="PASSENGER-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>PASSENGER</a></span></li><li><span><a href="#SEAT" data-toc-modified-id="SEAT-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>SEAT</a></span></li><li><span><a href="#DATE" data-toc-modified-id="DATE-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>DATE</a></span></li></ul></li><li><span><a href="#Fact-table-:-NOTES" data-toc-modified-id="Fact-table-:-NOTES-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fact table : NOTES</a></span><ul class="toc-item"><li><span><a href="#Service" data-toc-modified-id="Service-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Service</a></span></li><li><span><a href="#Passenger" data-toc-modified-id="Passenger-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Passenger</a></span></li><li><span><a href="#Route" data-toc-modified-id="Route-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Route</a></span></li><li><span><a href="#Date" data-toc-modified-id="Date-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Date</a></span></li><li><span><a href="#Seat" data-toc-modified-id="Seat-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Seat</a></span></li><li><span><a href="#Airline" data-toc-modified-id="Airline-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Airline</a></span></li><li><span><a href="#Deletion" data-toc-modified-id="Deletion-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Deletion</a></span></li><li><span><a href="#Modification-of-&quot;Overall_Customer_Rating&quot;-parameter" data-toc-modified-id="Modification-of-&quot;Overall_Customer_Rating&quot;-parameter-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Modification of "Overall_Customer_Rating" parameter</a></span></li><li><span><a href="#Columns-reordering" data-toc-modified-id="Columns-reordering-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>Columns reordering</a></span></li></ul></li></ul></div>

## Introduction

***Import***

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import unicodedata
import re
from tqdm import tqdm

***Path***

In [2]:
# (!) Change the path
path = "/home/theo/Documents/S6/Projet_InterPromo/Donnees/All Data/"

***Converting files to Dataframe***

In [3]:
aircraft_detail = pd.read_excel(path + "AIRCRAFT_DETAIL_V2.xlsx")
all_data = pd.read_excel(path + "ALL_DATA.xlsx")
all_data_ann = pd.read_excel(path + "ALL_DATA_ANNOTATE.xlsx")
data_kayak_route = pd.read_excel(path + "DATA_KAYAK_ROUTE.xlsx")
kayak_airline_route = pd.read_excel(path + "KAYAK_AIRLINE_RATE_GLOBAL.xlsx")
others_data = pd.read_excel(path + "OTHERS_DATA_ANNOTATE.xlsx")
seat_info = pd.read_csv(path + "SEATGURU_INFO_AIRCRAFT.csv", sep=";")

***Connection to the MySQL database***

In [4]:
# (!) Change the MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="29101999esg",
    database="BDD"
)

mycursor = mydb.cursor()

## Tables dimension

###  AIRLINE

***Copying the file***

In [5]:
to_insert_in_airline = kayak_airline_route.copy()

***Delete column***

In [6]:
del to_insert_in_airline['Data_Source']

***List***

In [7]:
#List of parameters in the "Airline" table
columns_airline = ['Boarding', 'Cabin_Staff_Service', 'Comfort',
                   'Entertainment', 'Food_And_Beverages', 'Overall_Airline_Rating']

***Functions***

In [8]:
# Replace values of columns
def COLUMNS_REPLACE_AIRLINE(df, columns_airline: list):
    """ Documentation
    Parameters :
        df : change dataframe
        columns_airline : list of columns to change    

    Out (if exists) :
        df : dataframe with replaced values and float columns    

    Runtime :
        12 ms ± 967 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
    """

    for i in range(len(columns_airline)):
        # replace "," with "." to set the columns to "float"
        df[columns_airline[i]] = df[columns_airline[i]].str.replace(',', '.')
        df[columns_airline[i]] = df[columns_airline[i]].apply(
            lambda x: float(x))

In [9]:
# Put a mention for each score
def FIND_MENTION(score: float):
    """ Documentation
    Parameters :
        score : score for airline   

    Out (if exists) :
        mention : str for the mention of airline

    Runtime :
        36.7 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    """

    # If the "score" is lower then we give the mention
    if score < 4:
        mention = "médiocre"
    elif score < 6:
        mention = "correct"
    elif score < 8:
        mention = "bien"
    else:
        mention = "excellent"

    return mention

***Program***

In [10]:
# Call the "COLUMNS_REPLACE_AIRLINE" function
COLUMNS_REPLACE_AIRLINE(to_insert_in_airline, columns_airline)

In [11]:
# Create a set with the airlines in the "kayak_airline_route" file
set_airline = set()

In [12]:
# Remove accents
to_insert_in_airline['Airline_Name'] = to_insert_in_airline['Airline_Name'].apply(lambda x: str(
    unicodedata.normalize('NFKD', x.lower()).encode('ascii', 'ignore'))[2:-1] if x is not None else x)

In [13]:
# If an airline is present several times, then the average of these evaluations is calculated
for airline in tqdm(to_insert_in_airline['Airline_Name']):
    df_Air = to_insert_in_airline[to_insert_in_airline['Airline_Name'] == airline]

    if len(df_Air) > 1:
        list_air = [airline,
                    np.mean(df_Air['Boarding']),
                    np.mean(df_Air['Cabin_Staff_Service']),
                    np.mean(df_Air['Comfort']),
                    np.mean(df_Air['Entertainment']),
                    np.mean(df_Air['Food_And_Beverages']),
                    np.mean(df_Air['Overall_Airline_Rating']),
                    FIND_MENTION(np.mean(df_Air['Overall_Airline_Rating'])),
                    sum(df_Air['Number_Evaluation'])]
        to_insert_in_airline = to_insert_in_airline[to_insert_in_airline['Airline_Name'] != airline]
        to_insert_in_airline = to_insert_in_airline.append(
            pd.DataFrame([list_air], columns=list(to_insert_in_airline)))

    # If an airline is not present, we add it to set
    if (not airline in set_airline):
        set_airline.add(airline)

100%|██████████| 400/400 [00:01<00:00, 342.45it/s]


In [14]:
# We add to set and "to_insert_in_airline" dataframe the airlines without notes (those of All_Data)
col_airline = all_data.copy()['Airline_Name'].apply(lambda x: str(unicodedata.normalize(
    'NFKD', x.lower()).encode('ascii', 'ignore'))[2:-1] if x is not None else x)

for new_airline in col_airline:

    if (not new_airline in set_airline):
        set_airline.add(new_airline)
        to_insert_in_airline = to_insert_in_airline.append(pd.DataFrame(np.array(
            [[new_airline, None, None, None, None, None, None, None, None]]), columns=list(to_insert_in_airline)))

In [15]:
# Replace "nan" by "None"
to_insert_in_airline[['Boarding', 'Cabin_Staff_Service', 'Comfort', 'Entertainment', 'Food_And_Beverages', 'Overall_Airline_Rating']] = to_insert_in_airline[[
    'Boarding', 'Cabin_Staff_Service', 'Comfort', 'Entertainment', 'Food_And_Beverages', 'Overall_Airline_Rating']].replace({pd.np.nan: None})

***Insertion in the database***

In [16]:
# We insert the values of "to_insert_in_airline" dataframe in the "AIRLINE" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_airline = "INSERT INTO AIRLINE (airline_name, boarding, cabin_staff_service, comfort, entertainment, food_and_beverages, overall_airline_rating, mention, number_evaluation) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

for i in tqdm(range(len(to_insert_in_airline))):
    line = list(to_insert_in_airline.iloc[i, :])
    if (line[8] is not None):
        line[8] = int(line[8])
    values_airline = (line[0], line[1], line[2], line[3],
                      line[4], line[5], line[6], line[7], line[8])
    mycursor.execute(sql_request_airline, values_airline)

mydb.commit()

100%|██████████| 727/727 [00:01<00:00, 718.63it/s]


### ROUTE

***Copying files***

In [17]:
kayak_route_df = data_kayak_route.copy()
kayak_all_data_df = all_data.copy()

***Fuctions***

In [18]:
# Thanks to the "dic_city" dictionary, we transform the name airport into the name of the city
def FIND_CITY(airport: str):
    """ Documentation
    Parameters :
        airport : name or code of aeroport    

    Out (if exists) :
        city : name of a city   

    Runtime :
        770 ns ± 31.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    """

    if '(' and ')' in airport:
        airport = airport[airport.find('(')+1:airport.find(')')]

        if len(airport) == 3 and airport in dic_city.keys():
            city = dic_city[airport.upper()]

        else:
            city = 'unknown'

    if len(airport) == 3 and airport in dic_city.keys():
        city = dic_city[airport.upper()]

    elif len(airport) == 3:
        city = 'unknown'

    else:
        city = airport
    city = unicodedata.normalize(
        'NFKD', city.lower()).encode('ascii', 'ignore')
    return city

***Dictionnary***

In [19]:
# Dictionnary with all cities and code
dic_city = {}

origine = kayak_route_df['Origine']
origine_code = kayak_route_df['Origine Code']
destination = kayak_route_df['Destination']
destination_code = kayak_route_df['Destination Code']

for i in tqdm(range(0, origine.size)):
    if (not origine_code[i].upper() in dic_city):
        dic_city[origine_code[i].upper()] = origine[i]

for i in tqdm(range(0, destination.size)):
    if (not destination_code[i].upper() in dic_city):
        dic_city[destination_code[i].upper()] = destination[i]

100%|██████████| 163171/163171 [00:02<00:00, 76772.71it/s]


***Program***

In [20]:
# Separate stages of the journey
# (ex : 'Pristina to Zürich via Ljubliana' -> origine: 'Pristina', destination: 'Zürich', via: 'Ljubliana')

list_route = kayak_all_data_df['Route']
list_route = list_route.dropna()
to_insert_in_route = pd.DataFrame(columns=['origine', 'destination', 'via'])
l_dep = [None]
l_arr = [None]
l_via = [None]

for route in tqdm(list_route):
    l_dep.append(FIND_CITY(route[:route.find(" to")]))

    if (route.find("via") != -1):
        l_arr.append(FIND_CITY(route[route.find(" to")+4:route.find(" via")]))
        l_via.append(FIND_CITY(route[route.find(" via")+5:]))
    else:
        l_arr.append(FIND_CITY(route[route.find(" to")+4:]))
        l_via.append(None)

to_insert_in_route['origine'] = l_dep
to_insert_in_route['destination'] = l_arr
to_insert_in_route['via'] = l_via

to_insert_in_route = to_insert_in_route.drop_duplicates()

100%|██████████| 61759/61759 [00:00<00:00, 109551.99it/s]


***Insertion in the database***

In [21]:
# We insert the values of "to_insert_in_route" dataframe in the "ROUTE" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_route = "INSERT INTO ROUTE (origine, destination, via) VALUES (%s, %s, %s)"

for i in tqdm(range(0, to_insert_in_route.shape[0])):
    line = list(to_insert_in_route.iloc[i, :])
    values_route = (line[0], line[1], line[2])
    mycursor.execute(sql_request_route, values_route)

mydb.commit()

100%|██████████| 33616/33616 [00:21<00:00, 1544.39it/s]


###  SERVICE

***Copying the file***

In [22]:
fieldnames_service = ['Category', 'Category_Detail']
to_insert_in_service = all_data[fieldnames_service].drop_duplicates().replace({
    pd.np.nan: None})

***Insertion in the database***

In [23]:
# We insert the values of "to_insert_in_service" dataframe in the "SERVICE" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_service = "INSERT INTO SERVICE (type_service, desc_service) VALUES (%s, %s)"

for i in tqdm(range(len(to_insert_in_service))):
    values_service = (
        to_insert_in_service.iloc[i, 0], to_insert_in_service.iloc[i, 1])
    mycursor.execute(sql_request_service, values_service)
mydb.commit()

100%|██████████| 2609/2609 [00:01<00:00, 2031.75it/s]


### PASSENGER

***Copying the file***

In [24]:
passenger_df = all_data.copy()

In [25]:
fieldnames_passenger = ['Type_Of_Lounge', 'Type_Of_Traveller']
to_insert_in_passenger = passenger_df[fieldnames_passenger].drop_duplicates().replace(
    regex=["Type Of Lounge ", " : \r\n : N/A :"], value="").replace({pd.np.nan: None})

***Insertion in the database***

In [26]:
# We insert the values of "to_insert_in_passenger" dataframe in the "PASSENGER" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_passenger = "INSERT INTO PASSENGER (type_of_lounge, type_of_traveller) VALUES (%s, %s)"

for i in tqdm(range(len(to_insert_in_passenger))):
    values_passenger = (
        to_insert_in_passenger.iloc[i, 0], to_insert_in_passenger.iloc[i, 1])
    mycursor.execute(sql_request_passenger, values_passenger)
mydb.commit()

100%|██████████| 12/12 [00:00<00:00, 267.32it/s]


### SEAT

***Copying the file***

In [27]:
seat_df = seat_info.copy()

***Delete columns***

In [28]:
del seat_df["Data_Source"]
del seat_df["Seat_Class"]
del seat_df["Rating"]

***Function***

In [29]:
# Extract the name of the manufacturer from the name of the aircraft
def NAME_MANUFACTURER(var: str):
    """ Documentation
    Parameters :
        var : name of the aircraft    

    Out (if exists) :
        name of the manufacturer    

    Runtime :
        392 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    """
    list_construct = ['Canadair', 'Dash', 'CRJ',  # Bombardier
                      'ERJ',  # Embraer
                      'irbus', 'boeing',
                      'Sukhoi', 'Tupolev', 'Ilyushin', 'Yakovlev', 'Beech', 'Avro', 'Aerospatiale', 'Airbus', 'Aircraft Industries - Let', 'Antonov', 'ATR', 'BAE SYSTEMS (Avro)', 'BAE SYSTEMS (BAC)', 'BAE SYSTEMS (HS)', 'Boeing', 'Boeing (McDonnell-Douglas)', 'Bombardier (Canadair)', 'Bombardier (de Havilland)', 'Bombardier (Shorts)', 'Britten-Norman Aircraft Ltd,' 'Canadian Vickers', 'COMAC', 'Embraer', 'Fairchild (Swearingen)', 'Fairchild Dornier GmbH', 'Fairchild-Hiller Corp', 'Fokker', 'General Dynamics Corp', 'Handley Page', 'Hawker Beechcraft Corp', 'Lockheed Aircraft Corp', 'Saab Aircraft AB', 'United Aircraft Corporation (Ilyushin)', 'United Aircraft Corporation (Sukhoi)', 'United Aircraft Corporation (Tupolev)', 'United Aircraft Corporation (Yakovlev)', 'Bombardier']

    for c in list_construct:
        if c in var:
            if c in ['Canadair', 'Dash', 'CRJ']:
                return 'Bombardier'
            elif c in ['ERJ']:
                return 'Embraer'
            elif c in ['irbus']:
                return 'Airbus'
            elif c in ['boeing']:
                return 'Boeing'
            else:
                return c
    return None

***List***

In [30]:
# Orgenising data in order
to_insert_in_seat = seat_df[['Airline_name', 'Aircraft_Type', 'Category', 'Seat_Type', 'Seat_Position', 'Flight_Type', 'Count', 'Total_seat', 'Recline', 'Premium', 'Pitch_minimum',
                             'Pitch_maximum', 'Width', 'Bedlength_maximum', 'Bedlength_minimum', 'Music', 'VideoType', 'Power_available', 'Power_kind', 'Usb_available', 'Usb_kind']]

***Program***

In [31]:
# Means of "Width" if there are 2 values
# (ex Width : 18-20 -> Width : 19 )

tempo = to_insert_in_seat[['Width']].copy()
for l in tqdm(range(tempo.shape[0])):
    case = str(tempo.iloc[l, 0]).replace(" ", "").replace("in", "").split("-")
    if len(case) == 2:
        tempo.iloc[l, 0] = (float(case[0]) + float(case[1])) / 2
to_insert_in_seat[['Width']] = tempo

100%|██████████| 10330/10330 [00:01<00:00, 8516.23it/s]


In [32]:
to_insert_in_seat = to_insert_in_seat.drop_duplicates()

In [33]:
# Put all data in lower
for col in tqdm(range(2, to_insert_in_seat.shape[1])):
    to_insert_in_seat.iloc[:, col] = to_insert_in_seat.iloc[:, col].apply(
        lambda x: x.lower() if type(x) is str else x)

100%|██████████| 19/19 [00:00<00:00, 80.03it/s]


In [34]:
to_insert_in_seat = to_insert_in_seat.replace(
    {pd.np.nan: None, 'NaN': None, 'nan': None, 'none': None})

In [35]:
# Thanks to the "Aircrafact_Type" column and the "NAME_MANUFACTURER" function, we create the "Manufacturer_Name" column
to_insert_in_seat.insert(1, "Manufacturer_Name", None)
to_insert_in_seat['Manufacturer_Name'] = to_insert_in_seat['Aircraft_Type'].apply(
    NAME_MANUFACTURER)

In [36]:
# We add a line made up of only zeros 
tempo = pd.DataFrame(np.array([[None, None, None, None, None, None, None, None, None, None, None, None,
                             None, None, None, None, None, None, None, None, None, None]]), columns=list(to_insert_in_seat))
to_insert_in_seat = pd.concat([to_insert_in_seat, tempo])

In [37]:
to_insert_in_seat = to_insert_in_seat.drop_duplicates(
    subset=['Airline_name', 'Aircraft_Type', 'Category', 'Seat_Position'])

***Insertion in the database***

In [38]:
# We insert the values of "to_insert_in_seat" dataframe in the "SEAT" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_seat = "INSERT INTO SEAT (airline_name, manufacturer_name, aircraft_type, category,seat_type, seat_position, flight_type, count, total_seat, recline, premium, pitch_min, pitch_max, width, bedlength_max, bedlength_min, music,video_type, power_available, power_kind, usb_available, usb_kind) VALUES (%s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s)"

for i in tqdm(range(len(to_insert_in_seat))):
    values_seat = list(to_insert_in_seat.iloc[i, :])
    if (values_seat[7] is not None):
        values_seat[7] = int(values_seat[7])
    if (values_seat[8] is not None):
        values_seat[8] = int(values_seat[8])
    mycursor.execute(sql_request_seat, values_seat)
mydb.commit()

100%|██████████| 5502/5502 [00:05<00:00, 1050.19it/s]


### DATE

***Functions***

In [39]:
# Test if leap year
def LEAP_YEAR(year: str):
    """ Documentation
    Parameters :
        year: test if leap year    

    Out (if exists) :
        bissextile: boolean value    

    Runtime :
        188 ns ± 2.93 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
    """

    # Initialize variable to false
    bissextile = False

    # If the year is divisible by 400
    if year % 400 == 0:
        bissextile = True
    # If the year is divisible by 100
    elif year % 100 == 0:
        bissextile = False
    # If the year is divisible by 4
    elif year % 4 == 0:
        bissextile = True

    return bissextile

In [40]:
# Insert the lines in the dataframe
def INSERT_DAY_MONTH(df, month: str, year: str):
    """ Documentation
    Parameters :
        month: List month or list leap month
        year: test if leap year    

    Out (if exists) :
        (insert the lines in the dataframe)    

    Runtime:
        2.42 s ± 630 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
     """

    # 12 months loop
    for m in (range(12)):
        # loop on the number of days in the month "m"
        for d in range(month[m]+1):
            # MONTH
            if m < 9:
                val_month = '0'+str(m+1)
            else:
                val_month = str(m+1)
            # DAY
            if d < 10:
                val_day = '0'+str(d)
            else:
                # Insert variables "day", "month" and "year" in the dataframe
                val_day = str(d)
            df.loc[len(df)] = ['00', val_day, val_month, str(year)]
            # Column "id_date": Concatenate the variables "day", "month" and "year"
            df['id_date'] = df['day'].str.cat(df['month'].values.astype(
                str)).str.cat(df['year'].values.astype(str))

In [41]:
# Add a new year to the DataFrame
def ADD_YEAR(df, year: list):
    """ Documentation
    Parameters :
        year: year to add    

    Out (if exists) :
        (insert the new year in the dataframe)

    Runtime:
        for 1 year :
        13.2 s ± 804 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    """

    # Loop on the year
    for y in tqdm(year):
        # Initialize variable
        list_month = month        # If the year is leap year
        if LEAP_YEAR(y) == True:
            list_month = leap_month        # Call the function "INSERT_DAY_MONTH"
        INSERT_DAY_MONTH(df, list_month, y)

***Lists***

In [42]:
# List of the number of days in a month
month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
leap_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

***Program***

In [43]:
# Create the dataframe "df"
to_insert_in_date = pd.DataFrame(columns=['id_date', 'day', 'month', 'year'])

In [44]:
# Create the line "None"
to_insert_in_date = pd.DataFrame(np.array([["99999999", "99", "99", "9999"]]), columns=[
    "id_date", "day", "month", "year"]).append(to_insert_in_date, ignore_index=True)

In [45]:
# YEAR of 1999 to 2020
year = list(range(1999, 2021))
# year = [2021]  # <- (!) Change item (for 1 year)

In [46]:
# Call the function "add_year"
ADD_YEAR(to_insert_in_date, year)

100%|██████████| 22/22 [02:39<00:00, 12.14s/it]


***Insertion in the database***

In [47]:
# We insert the values of "to_insert_in_date" dataframe in the "DATE" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_date = "INSERT INTO DATE VALUES (%s, %s, %s, %s)"
for i in tqdm(range(len(to_insert_in_date))):
    values_date = list(to_insert_in_date.iloc[i, :])
    mycursor.execute(sql_request_date, values_date)
mydb.commit()

100%|██████████| 8301/8301 [00:05<00:00, 1582.90it/s]


## Fact table : NOTES

***Copying the file***

In [48]:
to_insert_in_note = all_data.copy()

In [49]:
to_insert_in_note = to_insert_in_note.replace({pd.np.nan: None})

### Service

In [50]:
# We realize a join (thanks to the Python function "merge") in order to link the "SERVICE" table to the "NOTES" table
sql_request_notes_service = "SELECT id_service, type_service, desc_service FROM SERVICE"
mycursor.execute(sql_request_notes_service)
notes_service = mycursor.fetchall()


tempo = pd.DataFrame(notes_service, columns=[
                     'id_service', 'Category', 'Category_Detail'])
to_insert_in_note = pd.merge(to_insert_in_note, tempo, on=[
                             'Category', 'Category_Detail'])

### Passenger

In [51]:
# We improve the "Type_Of_Lounge" column in order that the data is comparable with that in the database
to_insert_in_note['Type_Of_Lounge'] = to_insert_in_note['Type_Of_Lounge'].replace(
    regex=["Type Of Lounge ", " : \r\n : N/A :"], value="")

In [52]:
# We realize a join (thanks to the Python function "merge") in order to link the "PASSENGER" table to the "NOTES" table 
sql_request_notes_passenger = "SELECT id_passenger, type_of_lounge, type_of_traveller FROM PASSENGER"
mycursor.execute(sql_request_notes_passenger)
notes_passenger = mycursor.fetchall()

tempo = pd.DataFrame(notes_passenger, columns=[
                     'id_passenger', 'Type_Of_Lounge', 'Type_Of_Traveller'])
to_insert_in_note = pd.merge(to_insert_in_note, tempo, on=[
                             'Type_Of_Lounge', 'Type_Of_Traveller'])

### Route

In [53]:
# Separate stages of the journey
# (ex: 'Pristina to Zürich via Ljubliana' -> origine: 'Pristina', destination: 'Zürich', via: 'Ljubliana')

list_route = kayak_all_data_df['Route']
route_df = pd.DataFrame(columns=['origine', 'destination', 'via'])
l_dep = []
l_arr = []
l_via = []

for route in list_route:
    if type(route) == str:
        if (route.find("via") != -1):
            l_dep.append(FIND_CITY(route[:route.find(" to")]))
            l_arr.append(
                FIND_CITY(route[route.find(" to")+4:route.find(" via")]))
            l_via.append(FIND_CITY(route[route.find(" via")+5:]))
        else:
            l_dep.append(FIND_CITY(route[:route.find(" to")]))
            l_arr.append(FIND_CITY(route[route.find(" to")+4:]))
            l_via.append(None)
    else:
        l_dep.append(None)
        l_arr.append(None)
        l_via.append(None)


route_df['origine'] = l_dep
route_df['destination'] = l_arr
route_df['via'] = l_via

In [54]:
# We gather "to_insert_in_note" and "route_df" dataframes
to_insert_in_note = pd.concat([to_insert_in_note, route_df], axis=1)

In [55]:
# We improve these columns in order that the data is comparable with that in the database
to_insert_in_note['origine'] = to_insert_in_note['origine'].apply(
    lambda x: str(x)[2:-1] if x is not None else x)
to_insert_in_note['destination'] = to_insert_in_note['destination'].apply(
    lambda x: str(x)[2:-1] if x is not None else x)
to_insert_in_note['via'] = to_insert_in_note['via'].apply(
    lambda x: str(x)[2:-1] if x is not None else x)

In [56]:
# We realize a join (thanks to the Python function "merge") in order to link the "ROUTE" table to the "NOTES" table
sql_request_notes_route = "SELECT id_route, origine, destination, via FROM ROUTE"
mycursor.execute(sql_request_notes_route)
notes_route = mycursor.fetchall()

tempo = pd.DataFrame(notes_route, columns=[
                     'id_route', 'origine', 'destination', 'via'])
to_insert_in_note = pd.merge(to_insert_in_note, tempo, on=[
                             'origine', 'destination', 'via'])

### Date

***Functions***

In [57]:
# Put all dates in the same format (id_date -> '00012019')
def ID_DATE(nb: int):
    """ Documentation
    Parameters :
        nb: element of the "all_data" table    

    Out (if exists) :
        new_nb: date in the correct format    

    Runtime:
        660 ns ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    """

    # Initialize variable
    new_nb = nb

    # If the length is 6
    if len(str(nb)) == 6:
        new_nb = "00"+new_nb
    # If the length is 7
    if len(str(nb)) == 7:
        new_nb = "0"+new_nb

    # If the type is str
    if type(nb) == str:
        # If the 5th character is '-' or '/' (ex : '2019-01-01')
        if nb[4] == '-' or nb[4] == '/':
            year = nb[:4]
            month = nb[5:7]
            day = nb[8:10]
            new_nb = str(day+month+year)

    return new_nb

In [58]:
# Replace items for 3 columns
def DATE(before: list, after: list):
    """ Documentation
    Parameters :
        before: list of items to be replaced
        after: list of items that will replace    

    Out (if exists) :
        all_data[['Date_Visit', 'Date_Flown', 'Date_Review']] : 3 columns of the dataframe    

    Runtime:
        3.46 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    """

    # Loop on the lists ("before" and "after")
    for i in tqdm(range(len(before))):
        to_insert_in_note[['Date_Visit', 'Date_Flown']] = to_insert_in_note[[
            'Date_Visit', 'Date_Flown']].replace(regex=[before[i]], value=after[i])
    return to_insert_in_note[['Date_Visit', 'Date_Flown']]

***Program***

In [59]:
# Lists for "date" function
before = ["1st January 1970", "January\ ", "February\ ", "March\ ", "April\ ", "May\ ", "June\ ", "July\ ",
          "August\ ", "September\ ", "October\ ", "November\ ", "December\ ", "th\ |st\ |rd\ |nd ", "\ \d{2}\:\d{2}\:\d{2}"]
after = ["99999999", "01", "02", "03", "04", "05",
         "06", "07", "08", "09", "10", "11", "12", "", ""]

# List for loop, to apply the "id_date" function
columns_date = ["Date_Visit", "Date_Flown"]

In [60]:
# Call the "date" function
to_insert_in_note[['Date_Visit', 'Date_Flown']] = DATE(before, after)

# Replace "nan" with "99999999"
to_insert_in_note[['Date_Visit', 'Date_Flown']] = to_insert_in_note[[
    'Date_Visit', 'Date_Flown']].replace({pd.np.nan: "99999999"})

100%|██████████| 15/15 [00:07<00:00,  3.69it/s]


In [61]:
# Loop for apply the "id_date" function on 3 columns
for i in tqdm(range(len(columns_date))):
    to_insert_in_note[columns_date[i]
                      ] = to_insert_in_note[columns_date[i]].apply(ID_DATE)

100%|██████████| 2/2 [00:00<00:00,  2.60it/s]


### Seat

***Function***

In [62]:
# Standardize aircraft names
def TRANSFORM(aircraft: str):
    """ Documentation
    Parameters :
        aircraft : name of the aircraft    

    Out (if exists) :
        aircraft : names transformed    

    Runtime:
        8.32 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
    """

    aircraft = str(aircraft)
    aircraft = aircraft.lower()
    aircraft = re.sub("(\s|-)", "", aircraft)
    aircraft = aircraft.replace('embraer', 'e')
    aircraft = aircraft.replace('ee', 'e')
    aircraft = aircraft.replace('airbus', 'a')
    aircraft = aircraft.replace('aa', 'a')
    aircraft = aircraft.replace('boeing', 'b')
    aircraft = aircraft.replace('bb', 'b')
    aircraft = re.sub("v[0-9].*$", "", aircraft)
    aircraft = re.sub(".+?(?=[a-z]{1}[0-9]{1,})", '', aircraft)
    aircraft = re.sub("(\().*$", "", aircraft)

    return(aircraft)

In [63]:
# Thanks to the "dict_class" dictionary, we determine the class of the seat
def FIND_CLASS(class_seat: str):
    """ Documentation
    Parameters :
        class_seat : name of the class of the seat of the "all_data" file    

    Out (if exists) :
        class_seat : name of the class    

    Runtime:
        279 ns ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    """
    try:
        class_seat = dict_class[class_seat]
    except:
        class_seat = None
    return class_seat

***Dictionary***

In [64]:
dict_class = {'Economy Class': 'economy',
              'Economy Comfort': 'economy',
              'Economy Privilege': 'economy',
              'Business Class': 'business',
              'Premium Class': 'premium',
              'Premium Economy': 'premium',
              'Premium': 'premium',
              'First Class': 'first'}

***Program***

In [65]:
sql_request_notes_seat = "SELECT id_seat, airline_name, aircraft_type, category, seat_position FROM SEAT"

mycursor.execute(sql_request_notes_seat)
notes_seat = mycursor.fetchall()

In [66]:
notes_seat = pd.DataFrame(notes_seat, columns=[
                          'id_seat', 'Airline_Name', 'Aircraft_Type', 'Cabin_Class', 'Seat_Position'])

In [67]:
# Thanks to the "TRANSFORM" function, the aircraft types are standardized
to_insert_in_note['Aircraft_Type'] = to_insert_in_note['Aircraft_Type'].apply(
    lambda x: TRANSFORM(x))
notes_seat['Aircraft_Type'] = notes_seat['Aircraft_Type'].apply(
    lambda x: TRANSFORM(x))

In [68]:
# Thanks to the "FIND_CLASS" function, the class of seat is with that in the database
to_insert_in_note['Cabin_Class'] = to_insert_in_note['Cabin_Class'].apply(
    lambda x: FIND_CLASS(x))
to_insert_in_note['Seat_Position'] = 'normal'
notes_seat = notes_seat.drop_duplicates(
    subset=['Airline_Name', 'Aircraft_Type', 'Cabin_Class', 'Seat_Position'])

In [69]:
# We realize a join (thanks to the Python function "merge") in order to link the "SEAT" table to the "NOTES" table 
to_insert_in_note = pd.merge(to_insert_in_note, notes_seat, on=[
                             'Airline_Name', 'Aircraft_Type', 'Cabin_Class', 'Seat_Position'], how='left')

In [70]:
to_insert_in_note['id_seat'] = to_insert_in_note['id_seat'].replace({
                                                                    pd.np.nan: None})

In [71]:
# We don't want null foreign keys, so we replace "None" with the "id_seat" having "None" values in the "Seat" table
sql_request_null_seat = "SELECT id_seat FROM SEAT WHERE airline_name IS NULL AND aircraft_type IS NULL AND category IS NULL AND seat_position IS NULL"

mycursor.execute(sql_request_null_seat)
null_seat = mycursor.fetchall()
index_null = null_seat[0][0]

to_insert_in_note['id_seat'] = to_insert_in_note['id_seat'].apply(
    lambda x: int(index_null) if x is None else int(x))

### Airline

In [72]:
# We improve the "Airline_Name" column in order that the data is comparable with that in the database
to_insert_in_note['Airline_Name'] = to_insert_in_note['Airline_Name'].apply(lambda x: str(
    unicodedata.normalize('NFKD', x.lower()).encode('ascii', 'ignore'))[2:-1] if x is not None else x)

### Deletion 

In [73]:
def delete(dataframe, columns_to_delete):
    for i in columns_to_delete:
        del dataframe[i]

In [74]:
# We delete the columns of "All_Data" which are in excess
columns_to_delete = ['Data_Source', 'Airline_Type', 'Aircraft_Type', 'Region_Operation', 'Cabin_Class', 'Type_Of_Lounge', 'Date_Review',
                     'Type_Of_Traveller', 'Airport', 'Route', 'Category', 'Category_Detail', 'Seat_Type', 'Review', 'Seat', 'origine', 'destination', 'via']

In [75]:
delete(to_insert_in_note, columns_to_delete)

### Modification of "Overall_Customer_Rating" parameter

In [76]:
to_insert_in_note['Overall_Customer_Rating'] = to_insert_in_note['Overall_Customer_Rating'].replace({
                                                                                                    'na': None})
to_insert_in_note['Overall_Customer_Rating'] = to_insert_in_note['Overall_Customer_Rating'].str.replace(
    ',', '.')

### Columns reordering

In [77]:
cols = ['Airline_Name', 'id_route', 'id_seat', 'Date_Visit', 'Date_Flown', 'id_passenger', 'id_service', 'Cabin_Staff_Service', 'Lounge_Staff_Service', 'Bar_And_Beverages', 'Food_And_Beverages', 'Ground_Service', 'Catering', 'Cleanliness', 'Lounge_Comfort', 'Aisle_Space',
        'Wifi_And_Connectivity', 'Inflight_Entertainment', 'Viewing_Tv_Screen', 'Power_Supply', 'Seat_Comfort', 'Seat_Legroom', 'Seat_Storage', 'Seat_Recline', 'Seat_Width', 'Washrooms', 'Value_For_Money', 'Overall_Customer_Rating', 'Overall_Service_Rating', 'Overall_Airline_Rating', 'Recommended']
to_insert_in_note = to_insert_in_note[cols]

***Insertion in the database***

In [78]:
# We insert the values of "to_insert_in_note" dataframe in the "NOTES" table thanks to the "mycursor" cursor established at the beginning of the Notebook
sql_request_notes = "INSERT INTO NOTES (`#airline_name`, `#id_route`, `#id_seat`, `#date_visit`, `#date_flown`, `#id_passenger`, `#id_service`, cabin_staff_service, lounge_staff_service, bar_and_beverages, food_and_beverages, ground_service, catering, cleanliness, lounge_comfort, aisle_space, wifi_and_connectivity, inflight_entertainment, viewing_tv_screen, power_supply, seat_comfort, seat_legroom, seat_storage, seat_recline, seat_width, washrooms, value_for_money, overall_customer_rating, overall_service_rating, overall_airline_rating, recommended) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

for i in tqdm(range(len(to_insert_in_note))):
    values_notes = list(to_insert_in_note.iloc[i, :])
    values_notes[1] = int(values_notes[1])
    values_notes[2] = int(values_notes[2])
    values_notes[5] = int(values_notes[5])
    values_notes[6] = int(values_notes[6])

    mycursor.execute(sql_request_notes, values_notes)

mydb.commit()

100%|██████████| 150453/150453 [01:49<00:00, 1377.24it/s]


In [79]:
mydb.close()